In [9]:
import pandas as pd
import numpy as np

import joblib
import lightgbm
from sklearn.metrics import ndcg_score

In [111]:
train_set = pd.read_csv('data/filled_training_set.csv')
# remain_features = ['prop_starrating', 'prop_review_score', 'prop_location_score2', 'promotion_flag']+['srch_id', 'prop_id'] + ['score']
# remain_features = ['prop_starrating', 'prop_review_score', 'prop_location_score2', 'promotion_flag']+['srch_id', 'score'] + ['prop_location_score1', 'prop_brand_bool', 'price_usd', 'prop_country_id','srch_query_affinity_score','visitor_hist_starrating', 'srch_room_count'] + ['comp_avg_rate_percent_diff']
# train_set = train_set[remain_features]
train_set = train_set.drop(columns=['prop_id', 'site_id', 'random_bool', 'random_bool', 'srch_destination_id', 'visitor_location_country_id'])

train_size = int(len(train_set) * 0.8)
train_df = train_set[:train_size]
validation_df  = train_set[train_size:]

In [112]:
qids_train = train_df.groupby("srch_id")["srch_id"].count().to_numpy()
X_train = train_df.drop(["srch_id", "score"], axis=1)
y_train = train_df["score"]

qids_validation = validation_df.groupby("srch_id")["srch_id"].count().to_numpy()
X_validation = validation_df.drop(["srch_id", "score"], axis=1)
y_validation = validation_df["score"]
print(qids_train)
print(qids_validation)

[28 32  5 ... 33 33  6]
[27 13 32 ... 24 28  6]


In [113]:
model = lightgbm.LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    verbosity=0,  # 设置verbosity参数来控制模型的详细程度
    num_leaves=64,  # 增加树的深度
    learning_rate=0.2,  # 增加学习率
    max_depth=-1,  # 树的最大深度，-1 表示不限制
    # min_child_samples=50,  # 叶子节点的最小样本数
    # 'subsample': 0.8,  # 数据子抽样率
    # 'colsample_bytree': 0.8,  # 特征子抽样率
    # lambda_l1=0.0,  # L1 正则化参数
    # lambda_l2=0.0,  # L2 正则化参数
    # 'max_depth': -1,  # 不限制树的深度
    # 'min_data_in_leaf': 50,  # 设置叶子节点的最小数据量
    # 'lambda_l1': 0.1,  # 增加L1正则化项
    # 'lambda_l2': 0.1,  # 增加L2正则化项
    # 'feature_fraction': 0.8,  # 设置特征抽样比例
    # 'bagging_fraction': 0.8,  # 设置数据抽样比例
    # 'bagging_freq': 5,  # 每5次迭代进行一次数据抽样
)

In [114]:
model.fit(
    X=X_train,
    y=y_train,
    group=qids_train,
    eval_set=[(X_validation, y_validation)],
    eval_group=[qids_validation],
    eval_at=5,
)

LGBMRanker(learning_rate=0.2, metric='ndcg', num_leaves=64,
           objective='lambdarank', verbosity=0)

In [115]:
ndcg_results = model.evals_result_['valid_0']['ndcg@5']
print(np.mean(ndcg_results))

0.3661091510710732


In [51]:
output = model.predict(X_train)
print(output)
ndcg = ndcg_score([y_train], [output])

print("NDCG:", ndcg)

[LightGBM] [Warning] lambda_l1 is set=0.0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0
[LightGBM] [Warning] lambda_l2 is set=0.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0
[-0.39257343 -1.06967374 -1.51226108 ... -0.93168773 -0.03693602
 -0.97134335]
NDCG: 0.8114963066198921


In [120]:
qids = train_df.groupby("srch_id")["srch_id"].count().to_numpy()
X = train_df.drop(["srch_id", "score"], axis=1)
y = train_df["score"]
print(qids)
model.fit(
    X=X,
    y=y,
    group=qids,
    eval_at=5,
)

[28 32  5 ... 33 33  6]
{}


In [123]:
output = model.predict(X)

0.8114963066198921


In [135]:
print(len(y))
print(len(output))
y_grouped = []
output_grouped = []
start = 0
for cnt in qids:
    end = start + cnt
    y_grouped.append(y[start:end])
    output_grouped.append(output[start:end])
    start = end

3966677
3966677


In [137]:
print(len(y_grouped))
print(len(output_grouped))

159784
159784
28    0.0
29    0.0
30    0.0
31    0.0
32    0.0
33    0.0
34    0.0
35    0.0
36    0.0
37    0.0
38    0.0
39    0.0
40    0.0
41    0.0
42    0.0
43    0.0
44    0.0
45    0.0
46    0.0
47    0.0
48    0.0
49    0.0
50    0.0
51    0.0
52    0.0
53    0.0
54    0.0
55    0.0
56    0.0
57    0.0
58    0.0
59    1.0
Name: score, dtype: float64


In [141]:
ndcg_results = []
for i in range(len(y_grouped)):
    ndcg = ndcg_score([y_grouped[i]], [output_grouped[i]], k=5)
    ndcg_results.append(ndcg)
    if i % 10000 == 0:
        print(f'Group {i}, ndcg:{ndcg}.')
# print(ndcg_results)
print(np.mean(ndcg_results))

Group 0, ndcg:0.0.
Group 10000, ndcg:0.0.
Group 20000, ndcg:0.38685280723454163.
Group 30000, ndcg:0.5.
Group 40000, ndcg:0.0.
Group 50000, ndcg:0.0.
Group 60000, ndcg:0.5.
Group 70000, ndcg:1.0.
Group 80000, ndcg:0.0.
Group 90000, ndcg:0.0754042070390954.
Group 100000, ndcg:0.0.
Group 110000, ndcg:0.5.
Group 120000, ndcg:0.0.
Group 130000, ndcg:0.38685280723454163.
Group 140000, ndcg:0.0.
Group 150000, ndcg:0.3500439500444236.
0.4052526836012198


In [143]:
joblib.dump(model, 'models/lightgbm.pkl')

['models/lightgbm.pkl']